In [ ]:
from collections import defaultdict
import numpy as np
import os
import json
import itertools
import networkx as nx
import pandas as pd
import random
import collections
from time import time
import xgi
import math
from tqdm import tqdm
import pickle as pk

def discrete_SIR(H,l,mu,theta,initial_infecteds=None,initial_recovereds=None,initial_number=1,tmin=0,tmax=float("Inf"),dt=1.0):
    #Simulates the discrete SIR model with threshold higher-order contagion on hypergraphs  
    #H - xgi.Hypergraph - The hypergraph on which we simulate the SIR contagion process
    #l - float - Infection probability of the model
    #mu - float - Healing rate
    #theta - float - Threshold for hyper-infection
    #initial_infecteds - list - Initially infected node IDs.
    #initial_recovereds -  list - Initially recovered node IDs.
    #initial_number - int - Number of initially infected nodes.
    #tmin - float - Time at which the simulation starts.
    #tmax - float - Time at which the simulation terminates (if it doean't reach the absorbin state)
    #dt - float - Time-step of the simulation.

    if initial_infecteds is None:
        initial_infecteds = random.sample(list(H.nodes), initial_number)

    if initial_recovereds is None:
        initial_recovereds = []

    status = defaultdict(lambda: "S")
    for node in initial_infecteds:
        status[node] = "I"
    for node in initial_recovereds:
        status[node] = "R"
    I = [len(initial_infecteds)]
    R = [len(initial_recovereds)]
    S = [H.num_nodes - I[0] - R[0]]
    
    times = [tmin]
    t = tmin

    new_status = status.copy()
    
    while t <= tmax and I[-1] != 0:
        t += dt
        
        S.append(S[-1])
        I.append(I[-1])
        R.append(R[-1])
        
        i_edge={}
        for edge_id in H.edges:
            edge = H.edges.members(edge_id)
            i_edge[edge_id]=0
            idx_s=[]
            for i in edge: #in each hyperedge estimate the number of I and the identitiy of nodes who are still S
                if status[i]=="I":
                    i_edge[edge_id]=i_edge[edge_id]+1
                elif new_status[i]=="S": 
                    idx_s.append(i)
            
            if len(idx_s)>0 and (i_edge[edge_id]>=math.ceil(theta*len(edge))) and (random.random() <= l*dt): #conditions for group infection in the considered hyperedge (hihger-order threshold)
                for i in idx_s:
                    new_status[i]="I" 
                I[-1] += len(idx_s)
                S[-1] += -len(idx_s)
                    
        for node in H.nodes: #recovery
            if status[node] == "I" and random.random() <= mu * dt:
                new_status[node] = "R"
                R[-1] += 1
                I[-1] += -1
        
        times.append(t)
        status = new_status.copy()        
            
    return np.array(times), np.array(S), np.array(I), np.array(R)

In [ ]:
dataset='congress-bills_simplices';name_file='{0}'.format(dataset);
#dataset='senate-bills_simplices';name_file='{0}'.format(dataset);
#dataset='senate-committees_simplices';name_file='{0}'.format(dataset);
#dataset='house-committees_simplices';name_file='{0}'.format(dataset);
#dataset='email-Enron_simplices'; name_file='{0}'.format(dataset); 
#dataset='email-Eu_simplices'; name_file='{0}'.format(dataset); 
#dataset='hyperedges-cat-edge-algebra-questions_simplices'; name_file='{0}'.format(dataset);
#dataset='hyperedges-cat-edge-geometry-questions_simplices'; name_file='{0}'.format(dataset);
#dataset='hyperedges-cat-edge-music-blues-reviews'; name_file='{0}'.format(dataset);
#dataset='Mid1'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='Elem1'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='InVS15'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='SFHH'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='LH10'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='LyonSchool'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='Thiers13'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='M_PL_015_ECO_ins';name_file='{0}'.format(dataset);
#dataset='M_PL_062_ECO_ins';name_file='{0}'.format(dataset);
#dataset='M_PL_015_ECO_pl';name_file='{0}'.format(dataset);
#dataset='M_PL_015_ECO_pl';name_file='{0}'.format(dataset);

with open('Data/{0}.json'.format(name_file)) as json_file:
    data = json.load(json_file)
    
data = [data[i] for i in range(len(data)) if len(data[i])>1] #select only interactions of size >=2
for i in range(len(data)): # sort each interaction with growing nodes' IDs (e.g. [2,1] -> [1,2])
    data[i].sort()
data.sort() # sort the interactions
data = list(data for data,_ in itertools.groupby(data)) # remove interactions duplicated
data.sort(key = len) # sort the interactions according to their length
size_max=len(data[-1])

H=xgi.Hypergraph(data)
N=len(H.nodes)
nodes=list(H.nodes)

## Simulation

In [ ]:
initial_infecteds=1
mu=0.1
n_runs = 300
l=0.001
theta=0.25

output_path = '{0}_corr_EFS_seed_tetha{1}_l{2}_runs{3}_mu{4}.csv'.format(dataset,theta,l,n_runs,mu)

f = open(output_path,'w')
f.write('id_seed,run,EFS\n')

for seed_id in nodes:
    for run_id in tqdm(range(n_runs)):
        y=discrete_SIR(H,l,mu,theta,initial_infecteds=[seed_id],initial_recovereds=None,initial_number=initial_infecteds,tmin=0,tmax=float(1e2),dt=1.0)
                
        line = "%i,%i,%i\n"%(seed_id,run_id,y[3][-1])
        f.write(line)
        
f.close()